In [1]:
import numpy as np
import time

import hoomd
from hoomd import md
import gsd.hoomd

from MyCustomForce import MyCustomForce

In [2]:
frame = gsd.hoomd.Frame()
frame.particles.types = ['A']
frame.particles.N = 1
frame.particles.typeid = [0]
frame.particles.position = [[1,0,0]]
L = 5
frame.configuration.box = [L, L, 0, 0, 0, 0]

with gsd.hoomd.open('./unbiased_sim_data/lattice.gsd', mode='w') as f:
    f.append(frame)

In [8]:
seed = np.random.randint(np.random.randint(10000))
cpu = hoomd.device.CPU(notice_level=2)
sim = hoomd.Simulation(device=cpu, seed=seed)
sim.create_state_from_gsd(filename='./unbiased_sim_data/lattice.gsd')

dt = 0.001
kT = 0.2
custom = MyCustomForce()
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                              kT=kT)

nl = md.nlist.Cell(buffer=0.4)

thermostats = md.methods.Brownian(filter=hoomd.filter.All(), 
                                    kT=kT)

thermostats.tally_reservoir_energy = True
thermostats.gamma.default = 1.0
thermostats.gamma_r.default = [1.0, 1.0, 1.0]

integrator = md.Integrator(dt=dt, methods=[thermostats], forces=[custom])

thermodynamic_properties = md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

In [ ]:
logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename=f'./unbiased_sim_data/unbiased_sim_traj.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             logger=logger,
                             mode='wb')

sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

for i in range(1000):
    sim.run(1000)
    gsd_writer.flush()